<a href="https://colab.research.google.com/github/peejay2/Projet-fin-formation-2/blob/main/RNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.8 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn.functional as F
from datasets import load_dataset
from torch.utils.data import DataLoader
import transformers
import torchtext
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AutoTokenizer
import torch.optim as optim
import numpy as np
import math
import re
from random import *
import numpy as np
import torch.nn as nn
import torch.optim as optim
import string
from collections import Counter

In [3]:
#load dataset
dataset = load_dataset("rotten_tomatoes")

# #load tokeniser - bert
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# #tokenise
# def tokenization(example):
#     return tokenizer(example["text"], padding="max_length")

# #apply function
# dataset = dataset.map(tokenization, batched=True)

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


In [6]:
#format data for usage in torch
dataset['train'].set_format(type='torch', columns=['text', 'label'])
dataset['validation'].set_format(type='torch', columns=['text', 'label'])
dataset['test'].set_format(type='torch', columns=['text', 'label'])

In [7]:
#rename columns for use in torch

tokenized_train = dataset['train'].rename_column("label", "labels")
tokenized_val = dataset['validation'].rename_column("label", "labels")
tokenized_train = tokenized_train.remove_columns("text")
tokenized_val = tokenized_val.remove_columns("text")

Preparing the words

In [8]:
from string import punctuation

print(punctuation)

# get rid of punctuation
all_text = dataset['train']['text']

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [9]:
len(' '.join(dataset['train']['text']))

980707

In [10]:
def tokenise(x):
  all_text = dataset[x]['text']
  # reviews_split = all_text.split('\n')
  all_text2 = ' '.join(all_text)

  # create a list of words
  words = all_text2.split()
  def remove_punctuation(s):
    return s.translate(str.maketrans("", "", string.punctuation))

# Remove punctuation from each string in the list
  cleaned_list = [remove_punctuation(item) for item in all_text]
  cleaned_list = [item for item in cleaned_list if item != ""]
  all_text = " ".join(cleaned_list)
  words = all_text.split()

  from collections import Counter

  ## Build a dictionary that maps words to integers
  counts = Counter(words)
  vocab = sorted(counts, key=counts.get, reverse=True)
  vocab_to_int = {word: ii for ii, word in enumerate(vocab,1)}

  ## use the dict to tokenize each review in reviews_split
  ## store the tokenized reviews in reviews_ints
  reviews_ints = []
  for review in cleaned_list:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

  return vocab_to_int, reviews_ints

      #for word in cleaned_list[review]])

In [11]:
vocab_to_int, tokenised_train = tokenise('train')
_, tokenised_val = tokenise('validation')
_, tokenised_test = tokenise('test')

In [12]:
# stats about vocabulary
# print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', tokenised_train[:1])


Tokenized review: 
 [[1, 654, 6, 2679, 5, 19, 1, 3140, 8924, 93, 6105, 3, 9, 297, 227, 5, 64, 2, 3141, 49, 2330, 32, 1409, 1867, 8925, 1410, 6106, 37, 889, 8926]]


In [57]:
for i in tokenised_train:
  if len(i) == 51:
    print(i, len(i))

[1, 16, 6, 1238, 5, 44, 28, 6206, 27, 1, 1563, 4761, 6207, 9, 94, 358, 43, 94, 27, 20, 1, 9097, 4, 216, 638, 12, 94, 35, 48, 319, 43, 86, 168, 29, 216, 767, 3, 9, 55, 6, 2, 3200, 3, 1429, 9098, 48, 351, 96, 208, 21, 27] 51
[10, 354, 4574, 237, 12, 439, 58, 39, 241, 4, 2, 67, 52, 395, 5, 2283, 212, 46, 3, 111, 10, 206, 34, 3003, 316, 19, 7, 274, 13795, 22, 1792, 3, 10, 120, 41, 19, 127, 3108, 5, 2, 1105, 7, 1, 756, 1052, 32, 2, 1105, 29, 1, 70] 51


In [72]:
for i, j in enumerate(tokenised_val):
  if len(j) >= 51:
    print(i,j)

In [78]:
dataset['train']['text'][320]


'clever , brutal and strangely soulful movie .'

Encoding

In [13]:
x = 0

for i in range(len(tokenised_val)):
  y = len(tokenised_val[i])
  if y > x:
    x = y
    idx = i
print(x, idx)

47 712


In [14]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's
        or truncated to the input seq_length.
    '''
    ## getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    ## for each review, I grab that review
    for i, row in enumerate(reviews_ints):
      features[i, -len(row):] = np.array(row)[:seq_length]

    return features

In [15]:
featurestest = pad_features(tokenised_test, seq_length=0)

In [16]:
# Test your implementation!

seq_length = 51

featurestrain = pad_features(tokenised_train, seq_length=seq_length)
featuresval = pad_features(tokenised_val, seq_length=seq_length)
featurestest = pad_features(tokenised_test, seq_length=seq_length)

## test statements - do not change - ##
assert len(featurestrain)==len(tokenised_train), "Your features should have as many rows as reviews."
assert len(featurestrain[0])==seq_length, "Each feature row should contain seq_length values."


# print first 10 values of the first 30 batches
print(featurestrain[:30,:10])

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


Create dataloader

In [17]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(featurestrain), dataset['train']['label'])
val_data = TensorDataset(torch.from_numpy(featuresval), dataset['validation']['label'])
test_data = TensorDataset(torch.from_numpy(featuresval), dataset['test']['label'])

In [18]:
batch_size = 50

In [19]:
train_loader = DataLoader(train_data, shuffle=True, batch_size=50,drop_last=True)
val_loader = DataLoader(val_data, shuffle=True, batch_size=50,drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=50,drop_last=True)

Model

In [20]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [21]:
class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                            dropout=drop_prob, batch_first=True)

        # dropout layer
        self.dropout = nn.Dropout(0.3)

        # linear and sigmoid layer
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # sigmoid function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels

        # return last sigmoid output and hidden state
        return sig_out, hidden


    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if(train_on_gpu):
          hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                   weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
          hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                   weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

        return hidden

In [83]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int) + 1 # +1 for zero padding + our word tokens
output_size = 1
embedding_dim = 768
hidden_dim = 512
n_layers = 3

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(18230, 768)
  (lstm): LSTM(768, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [84]:
# loss and optimization functions
lr=0.0001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [85]:
net.zero_grad()



In [86]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# ... (define SentimentRNN class and related code)

# Training parameters
epochs = 10
counter = 0
print_every = 100
clip = 5

# Initialize accuracy lists outside the loop
train_acc_list = []
val_acc_list = []

# Move model to GPU, if available
if train_on_gpu:
    net.cuda()

net.train()

# Train for some number of epochs
for e in range(epochs):
    # ... (previous code)

    train_correct = 0
    train_total = 0
    h = net.init_hidden(batch_size)

    for inputs, labels in train_loader:
        counter += 1

        if train_on_gpu:
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state
        h = tuple([each.data for each in h])

        # Zero accumulated gradients
        net.zero_grad()

        # Get the output from the model
        output, h = net(inputs, h)

        # Calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # Calculate training accuracy
        predicted = (output > 0.5).float()  # Convert to 0 or 1
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

        # Loss stats
        if counter % print_every == 0:
            # ... (previous code)

            # Calculate validation accuracy
            val_h = net.init_hidden(batch_size)
            val_losses = []
            val_correct = 0
            val_total = 0

            for inputs, labels in val_loader:
                # ... (previous code)

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()


                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

                val_total += labels.size(0)
                val_correct += (output > 0.5).float().eq(labels).sum().item()  # Compare outputs to labels

            val_accuracy = (val_correct / val_total) * 100
            train_accuracy = (train_correct / train_total) * 100
            val_acc_list.append(val_accuracy)
            train_acc_list.append(train_accuracy)

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Train Accuracy: {:.2f}%".format(train_accuracy),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)),
                  "Val Accuracy: {:.2f}%".format(val_accuracy))


Epoch: 1/10... Step: 100... Loss: 0.707839... Train Accuracy: 54.56% Val Loss: 0.710369 Val Accuracy: 53.71%
Epoch: 2/10... Step: 200... Loss: 0.455444... Train Accuracy: 70.13% Val Loss: 0.784244 Val Accuracy: 52.76%
Epoch: 2/10... Step: 300... Loss: 0.410933... Train Accuracy: 71.15% Val Loss: 0.799940 Val Accuracy: 52.10%
Epoch: 3/10... Step: 400... Loss: 0.442471... Train Accuracy: 81.60% Val Loss: 1.014233 Val Accuracy: 51.90%
Epoch: 3/10... Step: 500... Loss: 0.293993... Train Accuracy: 80.36% Val Loss: 0.937058 Val Accuracy: 52.48%
Epoch: 4/10... Step: 600... Loss: 0.286251... Train Accuracy: 88.00% Val Loss: 1.278196 Val Accuracy: 53.14%
Epoch: 5/10... Step: 700... Loss: 0.171852... Train Accuracy: 94.20% Val Loss: 1.722899 Val Accuracy: 53.90%
Epoch: 5/10... Step: 800... Loss: 0.115765... Train Accuracy: 92.92% Val Loss: 1.696099 Val Accuracy: 53.71%
Epoch: 6/10... Step: 900... Loss: 0.148831... Train Accuracy: 96.16% Val Loss: 1.957957 Val Accuracy: 52.29%
Epoch: 6/10... Step

In [ ]:
val_acc_list

[51.33333333333333]

On test data

In [87]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

test_losses = []  # track loss
num_correct = 0
num_true_positives = 0
num_actual_positives = 0

# Initialize lists to store predictions and labels
all_predictions = []
all_labels = []

# init hidden state
h = net.init_hidden(batch_size)

net.eval()

# iterate over test data
for inputs, labels in test_loader:
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if train_on_gpu:
        inputs, labels = inputs.cuda(), labels.cuda()

    # get predicted outputs
    output, h = net(inputs, h)

    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())

    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer

    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

    # Collect predictions and labels
    all_predictions.extend(pred.cpu().tolist())
    all_labels.extend(labels.cpu().tolist())

    # Calculate True Positives and Actual Positives
    true_positives = (pred == 1) & (labels == 1)
    num_true_positives += true_positives.sum().item()
    num_actual_positives += (labels == 1).sum().item()

# Calculate Recall
recall = num_true_positives / num_actual_positives

# Calculate evaluation metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions)

# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# print evaluation metrics
print("Test accuracy:", accuracy)
print("Test precision:", precision)
print("Test recall:", recall)
print("Test F1-score:", f1)

Test loss: 3.044
Test accuracy: 0.539047619047619
Test precision: 0.53584229390681
Test recall: 0.5706106870229007
Test F1-score: 0.5526802218114603


tensor([1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1.,
        0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0.,
        1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1.],
       grad_fn=<RoundBackward0>)

In [ ]:
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'


In [ ]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuatuon
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review(test_review_neg)
print(test_ints)

seq_length = 200
features = pad_features(test_ints, seq_length)

print(features)

# test conversion to tensor and pass it to model
feature_tensor = torch.from_numpy(features)
print(feature_tensor.size())

[[1, 352, 18, 39, 35, 197, 209, 97, 970, 3, 39, 240, 173, 1054, 222, 14, 18, 199, 76, 209, 3, 1, 232, 97, 480]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0   

In [ ]:
def predict(net, test_review, sequence_length=200):
    ''' Prints out whether a give review is predicted to be
        positive or negative in sentiment, using a trained model.

        params:
        net - A trained net
        test_review - a review made of normal text and punctuation
        sequence_length - the padded length of a review
        '''

    net.eval()

    # tokenize review
    test_ints = tokenize_review(test_review)

    # pad tokenize sequence
    seq_length = sequence_length
    features = pad_features(test_ints, seq_length)

    # convert to tensor to pass to model
    feature_tensor = torch.from_numpy(features)

    batch_size = feature_tensor.size(0)

    # initialize hidden state
    h = net.init_hidden(batch_size)

    if(train_on_gpu):
      feature_tensor = feature_tensor.cuda()

    # get the output from the model
    output, h = net(feature_tensor, h)

    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))

    # print custom response based on whether test_review is pos/neg
    if(pred.item()==1):
      print('Positive review detected!')
    else:
      print('Negative review detected!')


In [ ]:
# positive test review
test_review_pos = 'Not good at all'


In [ ]:
# call function
# try negative and positive reviews!
seq_length=200
predict(net, test_review_neg, seq_length)
predict(net, test_review_pos, seq_length)

Prediction value, pre-rounding: 0.001106
Negative review detected!
Prediction value, pre-rounding: 0.948049
Positive review detected!
